### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__気象庁 震度データベース検索__  
  URL:  
　・https://www.data.jma.go.jp/svd/eqdb/data/shindo/index.html  
  DATA:   
　・地震リスト.csv  
  データ抽出方法:  
  　以下の条件で絞りこんでCSVダウンロードします。

  ・検索順「発生日時の古い順」  
  ・地震の発生日時・最大震度  
　  - 地震の発生日時「全期間」  
　  - 「最大震度１以上」  
  ・観測された震度  
　  - 都道府県：東京都  
　  - 市町村：無条件  
　  - 観測点：無条件  
　  - 上記で「震度４以上」を観測  
  ダウンロード方法:  
  　ページ右上にある三本線のアイコンをクリックしてデータの一覧を表示し、一覧の末尾の「CSVダウンロード」を押します。

In [ ]:
%matplotlib inline

import branca.colormap as cm
from decimal import Decimal, ROUND_HALF_UP
import folium
from geopy.distance import geodesic
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import seaborn as sns

### 1. データ型の変換

地震データをロード。

In [ ]:
earthquake_tokyo_df = pd.read_csv('data/地震リスト.csv')

len(earthquake_tokyo_df)

中身を確認。

どんなデータがありますか？

In [ ]:
earthquake_tokyo_df.head(20)

データの型と欠損を確認。

In [ ]:
earthquake_tokyo_df.info()

#### 1-1. 日時型への変換

まずは日付の変換をします。

`地震の発生日`は日時型ではないようですので日付のフォーマットを指定して日時型に変換し、新たな列の`occurance_date`に格納します。

参考）  
strftime() と strptime() の振る舞い  
https://docs.python.org/ja/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
try:
    earthquake_tokyo_df['occurance_date'] = pd.to_datetime(earthquake_tokyo_df['地震の発生日'], format="%Y/%m/%d")
except ValueError as e:
    print(e)

変換でエラーが発生していまいました。

フォーマットに問題があるようですが、まとめて確認したいので、引数`errors='coerce'`を指定して再実行します。

参考）  
pandas.to_datetime  
https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [ ]:
earthquake_tokyo_df['occurance_date'] = pd.to_datetime(earthquake_tokyo_df['地震の発生日'], format="%Y/%m/%d", \
                                                       errors='coerce')

earthquake_tokyo_df['occurance_date']

変換失敗数を確認。

失敗はNA（Not Assigned）であるかで判断します。

In [ ]:
print("「地震の発生日」のNA数:", earthquake_tokyo_df['地震の発生日'].isna().sum())
print("「occurance_date」のNA数:", earthquake_tokyo_df['occurance_date'].isna().sum())

失敗があるようなので、どんなデータで失敗しているかを確認。

In [ ]:
earthquake_tokyo_df[earthquake_tokyo_df['occurance_date'].isna()]['地震の発生日']

In [ ]:
earthquake_tokyo_df

フォーマットが分かったので、個々のデータの型で判定して失敗したものだけを再変換します。

手順としては以下のようになります。
1. 各行を引数として受け取る関数を作成し、欠損値の場合は`地震の発生日`列のデータを変換、そうでない場合はそのままの値を返す。

2. 作成した関数を`apply()`に引数として渡して実行する。

まずは各行を引数として受け取る関数を作成します。

欠損値の判定には`pd.isnull()`を利用します。

In [ ]:
### チャレンジしてみましょう！！ ###

作成した関数と`axis=1`を`apply()`に引数として渡して実行します。

`axis=1`を指定することで行ごとに処理が実行されるようになります。

In [ ]:
### チャレンジしてみましょう！！ ###

無名関数を渡して`apply()`を実行することも可能です。

In [ ]:
earthquake_tokyo_df['occurance_date'] = earthquake_tokyo_df.apply(
    lambda x: pd.to_datetime(x['地震の発生日'], format="%Y年%m月") if pd.isnull(x['occurance_date']) \
                else x['occurance_date'], axis=1)

earthquake_tokyo_df['occurance_date']

もう一度NAがないか確認。

In [ ]:
earthquake_tokyo_df['occurance_date'].isna().sum()

#### 1-2. 数値型への変換

次に`Ｍ`（マグニチュード）を`np.float64`型を指定して変換し、新たな列`magnitude`に格納します。

`astype()`にPythonの`float`型を指定することもできますが、その場合は自動で環境に合わせた`dtype`に変換されます。

In [ ]:
try:
    earthquake_tokyo_df['magnitude'] = earthquake_tokyo_df['Ｍ'].astype(np.float64)
except ValueError as e:
    print(e)

「不明」というデータが混ざっているため、エラーとなってしまいました。

「不明」を`np.nan`にしてから変換してみます。

In [ ]:
earthquake_tokyo_df['magnitude'] = earthquake_tokyo_df['Ｍ'].replace('不明', np.nan).astype(np.float64)

earthquake_tokyo_df['magnitude']

#### 1-3. 文字列型からカテゴリ型への変換

今度は集計やグラフで使いやすいよう、`最大震度`をカテゴリ型に変換します。

カテゴリ数が少ない場合、カテゴリ型に変換することでデータのサイズを小さくすることができます。

どんなデータがどの程度入っているか確認。

In [ ]:
earthquake_tokyo_df['最大震度'].value_counts()

数も少なく、カテゴリ分けするには丁度よさそうな感じです。

カテゴリ型に変換し、新しい列`max_intensity`に格納します。

In [ ]:
earthquake_tokyo_df['max_intensity'] = earthquake_tokyo_df['最大震度'].astype('category')

earthquake_tokyo_df['max_intensity']

カテゴリ型に変換はできたようなのですが、カテゴリの順序に少々問題があります。  
震度は階級なので、順番を考慮する必要があります。

参考）  
気象庁 震度について  
https://www.jma.go.jp/jma/kishou/know/shindo/index.html

In [ ]:
earthquake_tokyo_df['max_intensity'].cat.categories

`reorder_categories()`で順番を整えます。

In [ ]:
earthquake_tokyo_df['max_intensity'] = earthquake_tokyo_df['max_intensity'].cat.reorder_categories( \
                                ['震度４', '震度５弱', '震度５', '震度５強', '震度６弱', '震度６', '震度６強', '震度７'])

earthquake_tokyo_df['max_intensity'].cat.categories

`max_intensity`列をソートすると、インデックスの順にソートされるようになりました。

In [ ]:
earthquake_tokyo_df['max_intensity'].sort_values()[495:505]

使い方にやや癖がありますが、`add_categories()`で元々データになかったカテゴリを追加することもできます。

参考）  
Appending new categories  
https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#appending-new-categories

In [ ]:
earthquake_tokyo_df['max_intensity'] = earthquake_tokyo_df['max_intensity'].cat.add_categories(['震度Ｘ'])

earthquake_tokyo_df['max_intensity'].cat.categories

カテゴリが存在する場合、グループ化した場合にデータがなくてもカテゴリがすべてインデックスとして出力されます。

In [ ]:
earthquake_tokyo_groupby_max_intensity = earthquake_tokyo_df.groupby('max_intensity') \
.agg(発生回数=('occurance_date', 'count'))

earthquake_tokyo_groupby_max_intensity

カテゴリの削除も同様にします。

In [ ]:
earthquake_tokyo_df['max_intensity'] = earthquake_tokyo_df['max_intensity'].cat.remove_categories(['震度Ｘ'])

earthquake_tokyo_df['max_intensity'].cat.categories

#### 1-4. 日時型からカテゴリ型への変換

最後に、年代ごとの地震の発生回数を分析しましょう。

地震の発生日時を10年単位の年代に区分けしてカテゴリを作成し、新しい列`occurance_decade`に格納します。

カテゴリの作成手順は以下です。

1. `occurance_date`から年を抽出
2. 10で割って小数点以下を切り捨て
3. 再び10を掛ける
4. カテゴリ型に変換

In [ ]:
earthquake_tokyo_df["occurance_decade"] = (earthquake_tokyo_df["occurance_date"].dt.year // 10 * 10) \
                                           .astype('category')

earthquake_tokyo_df.head(40)

では年代、最大震度でグループ化し、地震の発生回数を取得します。

In [ ]:
earthquake_tokyo_group_by_decade = earthquake_tokyo_df \
                                    .groupby(['occurance_decade', 'max_intensity']) \
                                    .agg(回数=('occurance_date', 'count'))

earthquake_tokyo_group_by_decade

グラフで表示してみましょう。

どんなことが読み取れますか？

In [ ]:
sns.set(style='whitegrid', font=['MS Gothic','Hiragino Sans']) # 日本語フォントは設定が必要

fig, ax = plt.subplots(figsize=(20, 10))

sns.lineplot(data=earthquake_tokyo_group_by_decade, x="occurance_decade", y="回数", hue="max_intensity")

ax.legend(loc="upper left", bbox_to_anchor=(1.01, 1.005))

### 2. 位置座標の変換

震央地を地図上で表示できるよう、位置座標の変換を行います。

まずはデータを確認。

In [ ]:
earthquake_tokyo_df[['緯度','経度']].head()

#### 2-1. 度分から度への変換

フォーマットと「度・分」表記であることが分かりました。

数値抽出のための正規表現を作り、列の中でマッチしないデータがないか確認します。

In [ ]:
regex = r'^(\d+)°(\d+\.\d)′\w$'

(~earthquake_tokyo_df['緯度'].str.match(regex)).sum()

マッチしないデータを確認。

In [ ]:
earthquake_tokyo_df[~earthquake_tokyo_df['緯度'].str.match(regex)][['緯度', '経度']].head()

「不明」以外ないか確認。

In [ ]:
earthquake_tokyo_df[earthquake_tokyo_df['緯度'] == '不明']['緯度'].count()

ないようなので、「不明」を`np.nan`に置き換えてから列の中の数値抽出をおこないます。

In [ ]:
latitude = (earthquake_tokyo_df['緯度'].replace('不明', np.nan).str.extract(regex)).astype(np.float64)
latitude.rename(columns={0:'deg', 1:'min'}, inplace=True)

latitude.head()

経度の数値も同じように抽出します。

In [ ]:
longitude = (earthquake_tokyo_df['経度'].replace('不明', np.nan).str.extract(regex)).astype(np.float64)
longitude.rename(columns={0:'deg', 1:'min'}, inplace=True)

longitude.head()

先に度分を度に変換する関数を定義します。

参考）  
Pythonで度と度分秒を相互に変換する  
https://www.odndo.com/posts/python-survey-deg-dms/

In [ ]:
def dm2deg(deg, min):
    """度分を度に変換。

    Args:
        deg (float64): 度。
        min (float64): 分。
        
    Returns:
        float64: 変換後の度。
    """
    
    return np.float64(Decimal(str(deg + (min / 60))).quantize(Decimal('0.0001'), rounding=ROUND_HALF_UP))

度分を度に変換し、新しい列`latitude`と`longitude`に格納します。

In [ ]:
earthquake_tokyo_df['latitude'] = latitude.apply(lambda x: dm2deg(x['deg'], x['min']), axis=1)
earthquake_tokyo_df['longitude'] = longitude.apply(lambda x: dm2deg(x['deg'], x['min']), axis=1)

earthquake_tokyo_df[['latitude', 'longitude']]

#### 2-2. 位置座標から二点間距離への変換

地球の表面上での最短距離を意味する「測地線距離」の計算手法はHubeny、Haversine、Vincenty、Karneyなど色々あるようですが、今回はKarneyで計算します。

参考）  
緯度経度から距離を算出するPythonのライブラリ ― GeoPy  
https://h-memo.com/python-geopy-distance/

試しに東京駅から新宿駅までの距離を計算してみます。

In [ ]:
# 東京駅座標
tokyo_station_coords = (35.6812, 139.7671)

# 新宿駅座標
shinjuku_station_coords = (35.6896, 139.7006)

print(f"{round(geodesic(tokyo_station_coords, shinjuku_station_coords).km, 2)}km")

なお、Googleマップで距離を確認するには、始点となる場所で右クリックをしてメニューを表示し「距離を測定」を選択、その後地図上で終点をクリックします。

参考）  
Googleマップ 東京駅  
https://maps.google.co.jp/maps?ll=35.6812,139.7671

In [ ]:
### チャレンジしてみましょう！！ ###

それでは各位置座標から東京駅までの距離を取得し、新たな列`distance_from_tokyo`に格納します。

まずは各行を引数として受け取る関数を作成します。

欠損値の判定には`pd.isnull()`を利用します。

In [ ]:
### チャレンジしてみましょう！！ ###

作成した関数と`axis=1`を`apply()`に引数として渡して実行します。

`axis=1`を指定することで行ごとに処理が実行されるようになります。

In [ ]:
### チャレンジしてみましょう！！ ###

無名関数を渡して`apply()`を実行することも可能です。

In [ ]:
earthquake_tokyo_df['distance_from_tokyo'] = earthquake_tokyo_df.apply(
    lambda x: np.nan if pd.isnull(x['latitude'])
                else round(geodesic(tokyo_station_coords, (x['latitude'], x['longitude'])).km, 2), axis=1)

earthquake_tokyo_df['distance_from_tokyo']

### 3. 地図上にデータを表示

ここでは、震央地を以下のルールに沿って地図上に表示してみます。
- 震度５弱以上が対象。
- 各震央地に円を配置。
- 東京での震度で色分けする。
- マグニチュードのエネルギーの大きさに合わせて円を大きくする。

対象となるデータを用意します。

In [ ]:
earthquake_tokyo_for_map_df = earthquake_tokyo_df[earthquake_tokyo_df['最大震度'] >= '震度５'] \
                                .dropna(subset=['latitude']) \
                                .sort_values('Ｍ', ascending=False) # 円が重なってもクリックできるようにする。

len(earthquake_tokyo_for_map_df)

次に震度の色分けの準備をします。

震度は階級なので、連続性のある色のセットを選びます。

ウィジェットを使いながら色を決めましょう。

In [ ]:
sns.choose_colorbrewer_palette('sequential'); # セミコロンで文字列の出力を抑える。

今回は赤にしておきます。

In [ ]:
# 16進数カラーコードを生成。
color_reds = sns.color_palette("Reds", len(earthquake_tokyo_for_map_df['max_intensity'].cat.categories)).as_hex()

# 色のセットを確認。
sns.palplot(color_reds)

最後に地震のエネルギーの大きさです。

普段よく使われるマグニチュードというのは実は対数値です。  
今回は実際のエネルギーの大きさに変換して利用し、その大きさを実感できるようにします。

地震が発するエネルギーの大きさを`E`（単位：ジュール）、マグニチュードを`M`とすると、次の関係があります。

$ \log_{10}E = 4.8 + 1.5M $

これを関数にしてみましょう。

In [ ]:
def magnitude_to_joules(magnitude):
    """マグニチュードをエネルギーの大きさ（ジュール）に変換。

    Args:
        magnitude (float64): マグニチュード。
        
    Returns:
        float64: エネルギーの大きさ（ジュール）。
    """
    return 10 ** (4.8 + 1.5 * magnitude)

この関数を使って各地震のエネルギーの大きさを取得し、新たな列`energy`に格納します。

In [ ]:
earthquake_tokyo_for_map_df['energy'] = earthquake_tokyo_for_map_df['magnitude'].apply(magnitude_to_joules)

earthquake_tokyo_for_map_df['energy']

それでは地図上に表示します。

どんなことが読み取れますか？

In [ ]:
max_intensity_list = earthquake_tokyo_for_map_df['max_intensity'].cat.categories.to_list()

map = folium.Map(location=tokyo_station_coords, zoom_start=5, tiles="cartodbpositron") # 地味な地図を選択。

for i, row in earthquake_tokyo_for_map_df.iterrows():

    popup_message_html = f"""
    <p>震央地名: {row['震央地名']}</p>
    <p>地震の発生日: {row['地震の発生日']}</p>
    <p>マグニチュード: {row['Ｍ']}</p>
    <p>深さ: {row['深さ']}</p>
    <p>東京都での最大震度: {row['最大震度']}</p>
    <p>東京都からの距離: {row['distance_from_tokyo']:,}km</p>
    """
    popup = folium.Popup(folium.IFrame(popup_message_html), min_width=400, max_width=400)
    
    color = color_reds[max_intensity_list.index(row['max_intensity'])]
    
    folium.Circle(location=(row['latitude'], row['longitude']),
                  radius=row['energy'] / 10**12.5,
                  color=color,
                  fill_color=color,
                  weight=1.5,
                  popup=popup
                 ).add_to(map)
    
display(map)

# 簡単なカラーバーを表示
print("", " ".join(earthquake_tokyo_for_map_df['max_intensity'].cat.categories))
sns.palplot(color_reds)